# Explore here

It's recommended to use this notebook for exploration purposes.

In [ ]:
#pip install pandas requests lxml

In [3]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd




In [4]:
url = "https://datosmacro.expansion.com/pib"
response = requests.get(url)
if response.status_code != 200:
    print(f"Error al acceder a la página: {response.status_code}")
    exit()
else:
    print("Conexion exitosa")

Conexion exitosa


In [5]:
response.text

'<!DOCTYPE html><html lang="es" dir="ltr" prefix="og: https://ogp.me/ns#"><head><meta charset="utf-8" />\n<meta name="description" content="El Producto Interior Bruto (PIB) mide el valor monetario de la producción de bienes y servicios finales de un país durante un año. El PIB también se define como el conjunto de todos los bienes y servicios finales producidos en un país durante un año." />\n<link rel="canonical" href="https://datosmacro.expansion.com/pib" />\n<meta property="og:site_name" content="Datosmacro.com" />\n<meta property="og:type" content="article" />\n<meta property="og:url" content="https://datosmacro.expansion.com/pib" />\n<meta property="og:title" content="PIB - Producto Interior Bruto 2025" />\n<meta property="og:description" content="El Producto Interior Bruto (PIB) mide el valor monetario de la producción de bienes y servicios finales de un país durante un año. El PIB también se define como el conjunto de todos los bienes y servicios finales producidos en un país du

In [8]:
soup = BeautifulSoup(response.text, "html.parser")
soup

<!DOCTYPE html>
<html dir="ltr" lang="es" prefix="og: https://ogp.me/ns#"><head><meta charset="utf-8"/>
<meta content="El Producto Interior Bruto (PIB) mide el valor monetario de la producción de bienes y servicios finales de un país durante un año. El PIB también se define como el conjunto de todos los bienes y servicios finales producidos en un país durante un año." name="description"/>
<link href="https://datosmacro.expansion.com/pib" rel="canonical"/>
<meta content="Datosmacro.com" property="og:site_name"/>
<meta content="article" property="og:type"/>
<meta content="https://datosmacro.expansion.com/pib" property="og:url"/>
<meta content="PIB - Producto Interior Bruto 2025" property="og:title"/>
<meta content="El Producto Interior Bruto (PIB) mide el valor monetario de la producción de bienes y servicios finales de un país durante un año. El PIB también se define como el conjunto de todos los bienes y servicios finales producidos en un país durante un año." property="og:description"

In [9]:
#Encontrar las tablas que quiero obtener
table = soup.find("table", class_="table tabledat table-striped table-condensed table-hover")

In [26]:
rows = table.find_all("tr")#buscar todas las filas "tr" y guardarlas en rows
celdas = table.find_all("td")
celdas



[<td><a href="/pib/espana">España [+]</a></td>,
 <td class="fecha" data-value="2024-12-01">2024</td>,
 <td class="numero eur" data-value="1594330">1.594.330 M€</td>,
 <td class="hbar eur wdsp1_2"><div class="graph_hbar" style="background-color: #F1BF00; width:5.9%;" title="1.594.330 M€"></div></td>,
 <td class="numero dol" data-value="1725672">1.725.672 M$</td>,
 <td class="hbar dol wdsp1_2"><div class="graph_hbar" style="background-color: #F1BF00; width:5.9%;" title="1.725.672 M$"></div></td>,
 <td class="numero" data-value="3.5">3,5%</td>,
 <td class="hbar wdsp1_2"><div class="graph_hbar" style="background-color: #F1BF00;margin-left: 37.9%; width:4.9%;" title="3,5%"></div></td>,
 <td><a href="/pib/alemania">Alemania [+]</a></td>,
 <td class="fecha" data-value="2024-12-01">2024</td>,
 <td class="numero eur" data-value="4328970">4.328.970 M€</td>,
 <td class="hbar eur"><div class="graph_hbar" style="background-color: #FFE000; width:16%;" title="4.328.970 M€"></div></td>,
 <td class="nu

In [42]:
datos = []

for row in rows:
    celdas = row.find_all("td")
    if len(celdas) < 7:   
        continue
#Extraer los datos de cada celda
##texto = cita.find("span", class_="text").get_text(strip=True)
    pais = celdas[0].get_text(strip=True)
    fecha = celdas[1].get_text(strip=True)
    pib = celdas[2].get_text(strip=True)
    var_pib = celdas[6].get_text(strip=True)  # en la web está en la 7ª celda


    datos.append({
        "pais": pais,
        "fecha": fecha,
        "pib": pib,
        "var_pib": var_pib})
datos

[{'pais': 'España [+]',
  'fecha': '2024',
  'pib': '1.594.330\xa0M€',
  'var_pib': '3,5%'},
 {'pais': 'Alemania [+]',
  'fecha': '2024',
  'pib': '4.328.970\xa0M€',
  'var_pib': '-0,5%'},
 {'pais': 'Reino Unido [+]',
  'fecha': '2024',
  'pib': '3.368.254\xa0M€',
  'var_pib': '1,1%'},
 {'pais': 'Francia [+]',
  'fecha': '2024',
  'pib': '2.921.412\xa0M€',
  'var_pib': '1,2%'},
 {'pais': 'Italia [+]',
  'fecha': '2024',
  'pib': '2.199.619\xa0M€',
  'var_pib': '0,7%'},
 {'pais': 'Portugal [+]',
  'fecha': '2024',
  'pib': '289.428\xa0M€',
  'var_pib': '2,1%'},
 {'pais': 'Zona Euro [+]',
  'fecha': '2024',
  'pib': '15.231.314\xa0M€',
  'var_pib': '0,9%'},
 {'pais': 'Estados Unidos [+]',
  'fecha': '2024',
  'pib': '27.076.281\xa0M€',
  'var_pib': '2,8%'},
 {'pais': 'Japón [+]',
  'fecha': '2024',
  'pib': '3.714.582\xa0M€',
  'var_pib': '0,1%'},
 {'pais': 'China [+]',
  'fecha': '2024',
  'pib': '17.327.917\xa0M€',
  'var_pib': '5,0%'},
 {'pais': 'Andorra [+]',
  'fecha': '2024',
  'pi

In [44]:
df = pd.DataFrame(datos)
print(df)

                pais fecha           pib var_pib
0         España [+]  2024  1.594.330 M€    3,5%
1       Alemania [+]  2024  4.328.970 M€   -0,5%
2    Reino Unido [+]  2024  3.368.254 M€    1,1%
3        Francia [+]  2024  2.921.412 M€    1,2%
4         Italia [+]  2024  2.199.619 M€    0,7%
..               ...   ...           ...     ...
194        Samoa [+]  2024        987 M€    9,4%
195        Yemen [+]  2022     22.331 M€    1,5%
196    Sudáfrica [+]  2024    370.661 M€    0,5%
197       Zambia [+]  2024     24.330 M€    4,0%
198     Zimbabue [+]  2024    600.988 M€    2,0%

[199 rows x 4 columns]
